In [1]:
# Load the related library
import numpy as np
import scipy
import time
import math
import pyarrow as pa
import pyarrow.parquet as pq
import dask_cudf
import cudf
import pandas as pd
import gc
import copy
import os
import sys

# # Parallel processing using modin

# import modin.pandas as pd
# import modin.config as cfg
# cfg.Engine.put('Dask')
# cfg.Memory.put(9000000000)
# # Ray
# import ray
# ray.init()

from pathlib import Path
from datetime import datetime, date, time, timedelta
from dateutil import relativedelta

# Dask is essential for lazy computation especially for low memory hardware 
import dask.dataframe as dd
from dask.distributed import Client
client = Client()
from dask.diagnostics import ProgressBar
pbar = ProgressBar()                
pbar.register()


2022-09-18 03:54:09,643 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-0gxl8b2n', purging
2022-09-18 03:54:09,643 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-qx1v_tji', purging
2022-09-18 03:54:09,644 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-x7h3t5e_', purging
2022-09-18 03:54:09,644 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-29se3qw1', purging


In [2]:
# Set max display dataframe
import pandas as pd
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)

In [3]:
# # If you are running on kaggle's notebook, this is important
# # Uncommment if you run this notebook on kaggle
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

### This code is inspired by :
1) xxxxyyyy80008 in the [Process Amex Train Data to Parquet Format](https://www.kaggle.com/code/xxxxyyyy80008/process-amex-train-data-to-parquet-format?scriptVersionId=100270604)
2) SOHAIL in the [csv_to_parquet,pickle amex](https://www.kaggle.com/code/sohailshaik272/csv-to-parquet-pickle-amex?scriptVersionId=105696793)

In [4]:
# Invoke the dask client
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.48 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39421,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.48 GiB
Comm: tcp://127.0.0.1:40101,Total threads: 2
Dashboard: http://127.0.0.1:44803/status,Memory: 3.87 GiB
Nanny: tcp://127.0.0.1:46521,


### Work plan
1) Read csv by chunk
2) Convert int64 to int32, float64 to float32, object to category

In [5]:
train_data = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/amex-default-prediction/train_data.csv'
test_data = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/amex-default-prediction/test_data.csv'
train_labels = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/amex-default-prediction/train_labels.csv'

In [6]:
train_loc = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2'
test_loc = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_test_V2'
train_labels_loc = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_train_labels_V2'


In [7]:
# Use dask instead of pandas to read csv

train_ddf = dd.read_csv(train_data)
test_ddf = dd.read_csv(test_data)
train_labels_ddf = dd.read_csv(train_labels)

In [8]:
# Inspect rows numbers

len(train_labels_ddf)

458913

In [9]:
len(test_ddf)

11363762

In [10]:
len(train_labels_ddf)

458913

In [11]:
train_labels_ddf.dtypes.to_list

<bound method IndexOpsMixin.tolist of customer_ID    object
target          int64
dtype: object>

In [12]:
test_ddf.dtypes.to_list

<bound method IndexOpsMixin.tolist of customer_ID     object
S_2             object
P_2            float64
D_39           float64
B_1            float64
B_2            float64
R_1            float64
S_3            float64
D_41           float64
B_3            float64
D_42           float64
D_43           float64
D_44           float64
B_4            float64
D_45           float64
B_5            float64
R_2            float64
D_46           float64
D_47           float64
D_48           float64
D_49           float64
B_6            float64
B_7            float64
B_8            float64
D_50           float64
D_51           float64
B_9            float64
R_3            float64
D_52           float64
P_3            float64
B_10           float64
D_53           float64
S_5            float64
B_11           float64
S_6            float64
D_54           float64
R_4            float64
S_7            float64
B_12           float64
S_8            float64
D_55           float64
D_56           floa

In [13]:
train_ddf.dtypes.to_list

<bound method IndexOpsMixin.tolist of customer_ID     object
S_2             object
P_2            float64
D_39           float64
B_1            float64
B_2            float64
R_1            float64
S_3            float64
D_41           float64
B_3            float64
D_42           float64
D_43           float64
D_44           float64
B_4            float64
D_45           float64
B_5            float64
R_2            float64
D_46           float64
D_47           float64
D_48           float64
D_49           float64
B_6            float64
B_7            float64
B_8            float64
D_50           float64
D_51           float64
B_9            float64
R_3            float64
D_52           float64
P_3            float64
B_10           float64
D_53           float64
S_5            float64
B_11           float64
S_6            float64
D_54           float64
R_4            float64
S_7            float64
B_12           float64
S_8            float64
D_55           float64
D_56           floa

In [14]:
# Inspect the object data

train_ddf[['customer_ID','S_2','D_64','D_63']].head()

,customer_ID,S_2,D_64,D_63
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,O,CR
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,O,CR
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,O,CR
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,O,CR
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,O,CR


In [15]:
# As per description, list of categorical data :
# ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
categorical_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
train_ddf[categorical_cols].head()

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68
0,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
1,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
2,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
3,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
4,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0


In [16]:
%%time

 chunksize = 200e3
 print('chunksize = ', chunksize)
 
def process_csv(chunk, file_loc):
    # convert float64 to float32
    float64_cols = chunk.select_dtypes(include=['float64']).columns.tolist()
    chunk[float64_cols] = np.float32(chunk[float64_cols].values)

    # convert int64 into int32
    int64_cols = chunk.select_dtypes(include=['int64']).columns.to_list()
    chunk[int64_cols] = np.int32(chunk[int64_cols].values)

    # save to parquet file
    table = pa.Table.from_pandas(chunk)
    pq.write_table(table, dest_file, compression='GZIP')

    del table, chunk
    gc.collect()

chunksize =  500000.0
CPU times: user 455 µs, sys: 0 ns, total: 455 µs
Wall time: 425 µs


In [17]:
%%time

# Use pandas instead of dask
# S_2 identified as datetime
# D_63, D_64 are str

dtypes = {'customer_ID': 'str',
       'D_63': 'str',
       'D_64': 'str'
       }

file = [train_data, test_data, train_labels]
loc = [train_loc, test_loc, train_labels_loc]

data = {file[i]: loc[i] for i in range(len(file))}
print('New dict is : \n'+str(data))


New dict is : 
{'/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/amex-default-prediction/train_data.csv': '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2', '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/amex-default-prediction/test_data.csv': '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_test_V2', '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/amex-default-prediction/train_labels.csv': '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_train_labels_V2'}
CPU times: user 214 µs, sys: 51 µs, total: 265 µs
Wall time: 208 µs


In [19]:
%%time

for file, loc in data.items() :
    os.mkdir(loc)
    with pd.read_csv(file, chunksize=chunksize, dtype=dtypes) as reader :
        for i, chunk in enumerate(reader):
            dest_file = f'{loc}/{i+1}.parquet'                
            print('\n',dest_file)
            process_csv(chunk, dest_file)


 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/1.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/2.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/3.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/4.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/5.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/6.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/7.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/8.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/9.parquet

 /mnt/b7b917e1-da9

In [18]:
%%time

# using modin pandas for parallel processing

for file, loc in data.items() :
    os.mkdir(loc)
    if data.values == train_labels_loc:
        with pd.read_csv(file, chunksize=chunksize, dtype=dtypes) as reader :
            for i, chunk in enumerate(reader):
                dest_file = f'{loc}/{i+1}.parquet'                
                print('\n',dest_file)
                process_csv(chunk, dest_file)
    else :
        with pd.read_csv(file, chunksize=chunksize, dtype=dtypes, parse_dates=['S_2']) as reader :
            for i, chunk in enumerate(reader):
                dest_file = f'{loc}/{i+1}.parquet'                
                print('\n',dest_file)
                process_csv(chunk, dest_file)


 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/1.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/2.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/3.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/4.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/5.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/6.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/7.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/8.parquet

 /mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train_V2/9.parquet

 /mnt/b7b917e1-da9

ValueError: Missing column provided to 'parse_dates': 'S_2'